In [1]:
import numpy as np
import pandas as pd
import gensim

from word_extractor.lexemes_vector import get_most_similar_lexemes, get_verctor_from_word, get_verctor_from_sense_key, get_word_from_vector, get_analogy
from word_extractor.wordnet import get_hypernum, get_hyponym, get_not_similar_hyponym
from utils.dataloader import read_lexemes_dict_and_list, read_mapping

input_words = ['car', 'toughness', 'velocity', 'tire']
synset_filepath = './data/synsets.txt'
lexemes_filepath = './data/lexemes.txt'
mapping_filepath = './data/mapping.txt'
word2vec_filepath = './data/GoogleNews-vectors-negative300.bin'

lexemes_dict, lexemes_list = read_lexemes_dict_and_list(lexemes_filepath)
mapping_dict = read_mapping(mapping_filepath)
word_embed = gensim.models.KeyedVectors.load_word2vec_format(word2vec_filepath, binary=True)

/Users/ryogo/.local/share/virtualenvs/keyword_generation-2Tlv01LJ/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
[nltk_data] Downloading package wordnet to /Users/ryogo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ryogo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
def show_word2vec_and_autoextend(word, synset_idx=0):
    print('####    word2vec     #####')
    for i, word_w2 in enumerate(word_embed.most_similar([word])):
        print(f'{word_w2[0]}')

    print('####    AutoExtend     #####')
    most_similar_lexemes = get_most_similar_lexemes(word, lexemes_dict, lexemes_list, mapping_dict, rank_range=10, is_single=False, synset_idx=synset_idx)
    for i, lexeme in enumerate(most_similar_lexemes[0]):
        print(f'{lexeme[0][:-18]}')

In [58]:
word_embed.most_similar(['cat'])

[('cats', 0.8099379539489746),
 ('dog', 0.760945737361908),
 ('kitten', 0.7464985251426697),
 ('feline', 0.7326234579086304),
 ('beagle', 0.7150582671165466),
 ('puppy', 0.7075453400611877),
 ('pup', 0.6934291124343872),
 ('pet', 0.6891531348228455),
 ('felines', 0.6755931973457336),
 ('chihuahua', 0.6709762215614319)]

In [59]:
word_embed.most_similar(['cat'])

[('cats', 0.8099379539489746),
 ('dog', 0.760945737361908),
 ('kitten', 0.7464985251426697),
 ('feline', 0.7326234579086304),
 ('beagle', 0.7150582671165466),
 ('puppy', 0.7075453400611877),
 ('pup', 0.6934291124343872),
 ('pet', 0.6891531348228455),
 ('felines', 0.6755931973457336),
 ('chihuahua', 0.6709762215614319)]

# 実験１ 単語実験

## 語義曖昧性

Suit

In [34]:
show_word2vec_and_autoextend('suit')

####    word2vec     #####
suits
lawsuit
Suit
Atta_chakki_delivery
lawsuits
countersuit
polka_dot_clown
His_showmanship_rhinestone
complaint
lawsuit_alleging
####    AutoExtend     #####
zoot_suit
garment
gabardine
tailcoat
tuxedo
suit
tux
suit
suit
trousers


In [23]:
show_word2vec_and_autoextend('chair')

####    word2vec     #####
chairs
Chair
chairperson
chairwoman
chairman
Vice_Chair
Co_Chair
chairing
Chairs
cochair
####    AutoExtend     #####
seat
sofa
armchair
stool
recliner
chaise_longue
swivel_chair
rocking_chair
chaise
lawn_chair


In [24]:
show_word2vec_and_autoextend('table', synset_idx=1)

####    word2vec     #####
tables
ConocoPhillips_BPAmerica
Capitalized_Included
tray
dining_room
banquette
rapping_cappella
sideboard
linen_tablecloth
Tables
####    AutoExtend     #####
worktable
table
bookcase
chair
room
sideboard
tray
workbench
furniture
credenza


## 上位下位

In [1]:
import random

import numpy as np
import nltk
from nltk.corpus import wordnet
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /Users/ryogo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
from word_extractor.lexemes_vector import AutoextendExtractor
from word_extractor.wordnet import get_hypernum, get_hyponym, get_not_similar_hyponym
import gensim
import random

lexemes_filepath = './data/lexemes.txt'
mapping_filepath = './data/mapping.txt'
autoextend = AutoextendExtractor(lexemes_filepath, mapping_filepath)

[nltk_data] Downloading package wordnet to /Users/ryogo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ryogo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [53]:
def calc_hypo_similarities(word, input_idx=0):
    synsets = wordnet.synsets(word)
    synset = synsets[input_idx]
    input_sensekey = synset.lemmas()[0].key()
    input_vector = autoextend.get_verctor_from_sense_key(input_sensekey)
    hyponyms = synset.hyponyms()
    similarities = []
    for hyponym in hyponyms:
        sense_key = hyponym.lemmas()[0].key()
        vector = autoextend.get_verctor_from_sense_key(sense_key)
        try:
            similarity = np.dot(input_vector, vector) / (np.linalg.norm(input_vector) * np.linalg.norm(vector))
        except TypeError:
            similarity = None
        similarities.append([sense_key, similarity])

    hypernum = synset.hypernym_paths()[0][-2]
    sense_key = hypernum.lemmas()[0].key()
    vector = autoextend.get_verctor_from_sense_key(sense_key)

    try:
        similarity = np.dot(input_vector, vector) / (np.linalg.norm(input_vector) * np.linalg.norm(vector))
    except TypeError:
        similarity = None
    similarities.append([sense_key, similarity])

    return similarities


In [54]:
calc_hypo_similarities('suit', input_idx=0)

[['business_suit%1:06:00::', None],
 ['double-breasted_suit%1:06:00::', None],
 ['pinstripe%1:06:02::', None],
 ['single-breasted_suit%1:06:00::', None],
 ['slack_suit%1:06:00::', None],
 ['zoot_suit%1:06:00::', 0.7350061260076879],
 ['garment%1:06:00::', 0.6561577242314361]]

In [55]:
calc_hypo_similarities('chair', input_idx=0)

[['armchair%1:06:00::', 0.7204521804987309],
 ['barber_chair%1:06:00::', 0.5998571239337049],
 ['chair_of_state%1:06:00::', None],
 ['chaise_longue%1:06:00::', 0.7061076524085895],
 ['eames_chair%1:06:00::', None],
 ['fighting_chair%1:06:00::', None],
 ['folding_chair%1:06:00::', 0.6344630272143017],
 ['highchair%1:06:00::', 0.6486443473993481],
 ['ladder-back%1:06:01::', None],
 ['lawn_chair%1:06:00::', 0.6627009266302227],
 ['rocking_chair%1:06:00::', 0.6950857865568983],
 ['straight_chair%1:06:00::', None],
 ['swivel_chair%1:06:00::', 0.6974390069294739],
 ['tablet-armed_chair%1:06:00::', None],
 ['wheelchair%1:06:00::', 0.581628379769692],
 ['seat%1:06:01::', 0.7591656969365312]]

In [56]:
calc_hypo_similarities('table', input_idx=1)

[['altar%1:06:01::', 0.5397057495133598],
 ['booth%1:06:02::', 0.394118120307541],
 ['breakfast_table%1:06:00::', None],
 ['card_table%1:06:01::', None],
 ['card_table%1:06:00::', None],
 ['coffee_table%1:06:00::', None],
 ['conference_table%1:06:00::', None],
 ['console_table%1:06:00::', None],
 ['counter%1:06:00::', 0.4434781322464155],
 ['desk%1:06:00::', 0.574331939682561],
 ['dressing_table%1:06:00::', None],
 ['drop-leaf_table%1:06:00::', None],
 ['gaming_table%1:06:00::', None],
 ['gueridon%1:06:00::', 0.4830200680404497],
 ['kitchen_table%1:06:00::', None],
 ['operating_table%1:06:00::', None],
 ['parsons_table%1:06:00::', None],
 ['pedestal_table%1:06:00::', None],
 ['pier_table%1:06:00::', None],
 ['platen%1:06:02::', 0.40077379332060414],
 ['pool_table%1:06:00::', None],
 ['stand%1:06:04::', 0.3542382843974022],
 ['table-tennis_table%1:06:00::', None],
 ['tea_table%1:06:00::', None],
 ['trestle_table%1:06:00::', None],
 ['worktable%1:06:00::', 0.7039313503114878],
 ['furnitu

In [15]:
word = 'chair'
synsets = wordnet.synsets(word)
synset = synsets[0]
hyponyms = synset.hyponyms()
hyponyms

[Synset('armchair.n.01'),
 Synset('barber_chair.n.01'),
 Synset('chair_of_state.n.01'),
 Synset('chaise_longue.n.01'),
 Synset('eames_chair.n.01'),
 Synset('fighting_chair.n.01'),
 Synset('folding_chair.n.01'),
 Synset('highchair.n.01'),
 Synset('ladder-back.n.01'),
 Synset('lawn_chair.n.01'),
 Synset('rocking_chair.n.01'),
 Synset('straight_chair.n.01'),
 Synset('swivel_chair.n.01'),
 Synset('tablet-armed_chair.n.01'),
 Synset('wheelchair.n.01')]

In [6]:
word = 'table'
synsets = wordnet.synsets(word)
synset = synsets[1]
hyponyms = synset.hyponyms()
hyponyms

[Synset('altar.n.01'),
 Synset('booth.n.01'),
 Synset('breakfast_table.n.01'),
 Synset('card_table.n.01'),
 Synset('card_table.n.02'),
 Synset('coffee_table.n.01'),
 Synset('conference_table.n.01'),
 Synset('console_table.n.01'),
 Synset('counter.n.01'),
 Synset('desk.n.01'),
 Synset('dressing_table.n.01'),
 Synset('drop-leaf_table.n.01'),
 Synset('gaming_table.n.01'),
 Synset('gueridon.n.01'),
 Synset('kitchen_table.n.01'),
 Synset('operating_table.n.01'),
 Synset('parsons_table.n.01'),
 Synset('pedestal_table.n.01'),
 Synset('pier_table.n.01'),
 Synset('platen.n.01'),
 Synset('pool_table.n.01'),
 Synset('stand.n.04'),
 Synset('table-tennis_table.n.01'),
 Synset('tea_table.n.01'),
 Synset('trestle_table.n.01'),
 Synset('worktable.n.01')]

### 単語提示実験準備

In [ ]:
from word_extractor.lexemes_vector import AutoextendExtractor
from word_extractor.wordnet import get_hypernum, get_hyponym, get_not_similar_hyponym
import gensim
import random

lexemes_filepath = './data/lexemes.txt'
mapping_filepath = './data/mapping.txt'
autoextend = AutoextendExtractor(lexemes_filepath, mapping_filepath)

In [13]:
def show_word2vec_and_hypo_hyper(word, synset_idx=0):
    # print('####    word2vec     #####')
    # for i, word_w2 in enumerate(word_embed.most_similar([word])):
    #     print(f'{word_w2[0]}')
    
    # print('####    word2vec + 上位下位     #####')
    # for i, word_w2 in enumerate(word_embed.most_similar([word])):
    #     if i < 6:
    #         print(f'{word_w2[0]}')
    #     else:
    #         break

    print(f'{get_hypernum(word, idx=synset_idx)[0]}')
    print(f'{get_hypernum(get_hypernum(word, idx=synset_idx)[0])[0]}')
    print(f'{get_hypernum(get_hypernum(get_hypernum(word, idx=synset_idx)[0])[0])[0]}')
    print(f'{get_hyponym(word, idx=synset_idx)[0]}')
    print(f'{get_hyponym(word, idx=synset_idx)[0]}')
    print(f'{get_hyponym(word, idx=synset_idx)[0]}')
    print(f'{get_hyponym(word, idx=synset_idx)[0]}')
    print(f'{get_hyponym(word, idx=synset_idx)[0]}')
    print(f'{get_hyponym(word, idx=synset_idx)[0]}')
    
    

In [14]:
show_word2vec_and_hypo_hyper('suit')

garment
clothing
consumer_goods
pinstripe
single-breasted_suit
business_suit
slack_suit
zoot_suit
business_suit


In [15]:
show_word2vec_and_hypo_hyper('table', synset_idx=1)

furniture
furnishing
accessory
card_table
worktable
stand
pedestal_table
console_table
tea_table


In [16]:
show_word2vec_and_hypo_hyper('dog', synset_idx=0)

canine
tooth
bone
pug
lapdog
lapdog
mexican_hairless
puppy
basenji


In [17]:
show_word2vec_and_hypo_hyper('chair', synset_idx=0)

seat
space
attribute
tablet-armed_chair
barber_chair
tablet-armed_chair
straight_chair
chair_of_state
wheelchair


## 類推

In [1]:
from word_extractor.lexemes_vector import AutoextendExtractor
from word_extractor.wordnet import get_hypernum, get_hyponym, get_not_similar_hyponym
import gensim
import random

lexemes_filepath = './data/lexemes.txt'
mapping_filepath = './data/mapping.txt'
autoextend = AutoextendExtractor(lexemes_filepath, mapping_filepath)

word2vec_filepath = './data/GoogleNews-vectors-negative300.bin'
word_embed = gensim.models.KeyedVectors.load_word2vec_format(word2vec_filepath, binary=True)


# 重複なし乱数
def rand_ints_nodup(a, b, k):
  ns = []
  while len(ns) < k:
    n = random.randint(a, b)
    if not n in ns:
      ns.append(n)
  return ns

[nltk_data] Downloading package wordnet to /Users/ryogo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ryogo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 修正用実験

In [54]:
import random

import numpy as np
from scipy.spatial import distance
import nltk
from nltk.corpus import wordnet 
nltk.download("wordnet")


class AutoextendExtractor():

    def __init__(self, lexemes_filepath, mapping_filepath):
        self.lexemes_filepath = lexemes_filepath
        self.mapping_filepath = mapping_filepath

        self.lexemes_dict, self.lexemes_list = self.read_lexemes_dict_and_list()
        self.mapping_dict = self.read_mapping()


    def get_most_similar_lexemes(self, word, rank_range=10, num_word=1, synset_idx=0, is_random=True):
        vector = self.get_verctor_from_word(word, synset_idx)
        if vector is not None:
            all_vectors = [lexeme[1] for lexeme in self.lexemes_list]
            cos_similarities = 1 - distance.cdist([vector], all_vectors, metric='cosine')[0]
            sorted_indices = np.argsort(cos_similarities)[::-1]
            most_similar_lexemes = []
            for i in range(rank_range):
                idx = sorted_indices[i+1] 
                most_similar_lexeme = self.lexemes_list[idx][0]
                cos_similarity = cos_similarities[idx]
                most_similar_lexemes.append([most_similar_lexeme, cos_similarity])

            output_lexemes = []
            if is_random:
                indices = self.rand_ints_nodup(0, rank_range-1, num_word)
            else:
                indices = range(num_word)
            for idx in indices:
                output_lexeme = most_similar_lexemes[idx][0][:-18]
                output_synset_name = most_similar_lexemes[idx][0][-17:]
                output_sense_keys = self.mapping_dict[output_synset_name]
                output_sense_key = None
                for sense_key_temp in output_sense_keys:
                    if output_lexeme in sense_key_temp.lower():
                        output_sense_key = sense_key_temp
                output_lexemes.append([output_lexeme, output_sense_key])
            return output_lexemes
        else:
            output = [[None, None] for _ in range(num_word)]
            return output


    def get_vector(self, input):
        if '%' in input:
            vector = self.get_verctor_from_sense_key(input)
        else:
            vector = self.get_verctor_from_word(input)

        return vector

    def get_verctor_from_word(self, word, synset_idx=0):
        try:
            # print(wordnet.synsets(word)[synset_idx].definition())
            sense_key = wordnet.synsets(word)[synset_idx].lemmas()[0].key()
            vector_key = self.get_key_from_value(self.mapping_dict, sense_key)
        
            lexemes_vector = self.lexemes_dict[f'{word}-{vector_key}']
            return lexemes_vector
        except (KeyError, IndexError, AttributeError):
            return None

    def get_verctor_from_sense_key(self, sense_key):
        try:
            word = sense_key[:-10]
            vector_key = self.get_key_from_value(self.mapping_dict, sense_key)
            lexemes_vector = self.lexemes_dict[f'{word}-{vector_key}']
            return lexemes_vector
        except (KeyError, IndexError, TypeError):
            return None
        
    def get_key_from_value(self, dic, val):
        for key, sense_keys in dic.items():
            for sense_key in sense_keys:
                if sense_key == val:
                    return key
        return None
    
    def get_senseky_from_word(self, word, synset_idx=0):
        try:
            sense_key = wordnet.synsets(word)[synset_idx].lemmas()[0].key()
            return sense_key
        except (KeyError, IndexError, AttributeError):
            return None

    def get_word_from_vector(self, vector, is_return_vector=False):
        all_vectors = [lexeme[1] for lexeme in self.lexemes_list]
        cos_similarities = 1 - distance.cdist([vector], all_vectors, metric='cosine')[0]
        
        sorted_indices = np.argsort(cos_similarities)[::-1]
        indices = sorted_indices[:10] 
        most_similar_lexemes = []
        for idx in indices:
            word = self.lexemes_list[idx][0][:-18]
            vector = all_vectors[idx].tolist()
            if is_return_vector:
                most_similar_lexemes.append([word, vector])
            else:
                most_similar_lexemes.append(word)
        
        return most_similar_lexemes

    def get_analogy(self, word1, word2, word3, is_single=True):
        word1_vec = self.get_vector(word1)
        word2_vec = self.get_vector(word2)
        word3_vec = self.get_vector(word3)

        if (word1_vec is not None) and (word2_vec is not None) and (word3_vec is not None):
            words = self.get_word_from_vector(word1_vec - word2_vec + word3_vec)
            output_word = None
            if is_single:
                for word in words:
                    if (word != word1) and (word != word2) and (word != word3):
                        output_word = word
                        break
                print(f'{word1} - {word2} + {word3} = {output_word}')
            else:
                output_word = words
            return output_word
        else:
            print('None')
            return None

    def get_analogy_and_vector(self, word1, word2, word3):
        word1_vec = self.get_vector(word1)
        word2_vec = self.get_vector(word2)
        word3_vec = self.get_vector(word3)

        if (word1_vec is not None) and (word2_vec is not None) and (word3_vec is not None):
            calculated_vec = word1_vec - word2_vec + word3_vec
            output_lexemes = self.get_word_from_vector(calculated_vec)
            word1_list = [[word1, word1_vec.tolist()]]
            word2_list = [[word2, word2_vec.tolist()]]
            word3_list = [[word3, word3_vec.tolist()]]
            cal_list = [[f'{word1}-{word2}+{word3}', calculated_vec.tolist()]]

            output = word1_list + word2_list + word3_list + cal_list + output_lexemes

            return output
        else:
            print('None')
            return None

    def get_analogy_by_sensekey(self, word1_sensekey, word2_sensekey, word3_sensekey):
        word1, word2, word3 = [word1_sensekey[:-10], word2_sensekey[:-10], word3_sensekey[:-10]]

        word1_vec = self.get_verctor_from_sense_key(word1_sensekey)
        word2_vec = self.get_verctor_from_sense_key(word2_sensekey)
        word3_vec = self.get_verctor_from_sense_key(word3_sensekey)

        output_word = None
        if (word1_vec is not None) and (word2_vec is not None) and (word3_vec is not None):
            words = self.get_word_from_vector(word1_vec - word2_vec + word3_vec)
            for word in words:
                if (word != word1) and (word != word2) and (word != word3):
                    output_word = word
                    break
            print(f'{word1} - {word2} + {word3} = {output_word}')
            return output_word
        else:
            print('None')
            return None

    def read_lexemes_dict_and_list(self):
        with open(self.lexemes_filepath) as f:
            lexemes_text = f.read().splitlines()
        lexemes_text = [i.split(' ') for i in lexemes_text][1:]
        lexemes_dict = {}
        lexemes_list = []
        for line in lexemes_text:
            lexemes_dict[line[0]] = np.array([np.float64(i) for i in line[1:]])
            lexemes_list.append([line[0], np.array([np.float64(i) for i in line[1:]])])
        return lexemes_dict, lexemes_list

    def read_mapping(self):
        with open(self.mapping_filepath) as f:
            mapping_text = f.read().splitlines()
        mapping_dict = {}
        for line in mapping_text:
            if len(line) == 18:
                synset_name = line[:-1]
                mapping_dict[synset_name] = ['']
            else:
                synset_name = line.split(' ')[0]
                sense_key = [lexeme for lexeme in (line.split(' ')[1]).split(',')][:-1]
                mapping_dict[synset_name] = sense_key
        return mapping_dict

    def rand_ints_nodup(self, a, b, k):
        ns = []
        while len(ns) < k:
            n = random.randint(a, b)
            if not n in ns:
                ns.append(n)
        return ns

[nltk_data] Downloading package wordnet to /Users/ryogo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [55]:
import gensim

lexemes_filepath = './data/lexemes.txt'
mapping_filepath = './data/mapping.txt'
autoextend = AutoextendExtractor(lexemes_filepath, mapping_filepath)

word2vec_filepath = './data/GoogleNews-vectors-negative300.bin'
word_embed = gensim.models.KeyedVectors.load_word2vec_format(word2vec_filepath, binary=True)

In [ ]:
gensim.models.KeyedVectors.most_similar

In [73]:
word_embed.most_similar('stag_beetle', topn=1000)

[('skink', 0.6500285267829895),
 ('kakariki', 0.6458336710929871),
 ('damselfish', 0.636304497718811),
 ('marsh_fritillary', 0.6337847113609314),
 ('ringed_plovers', 0.6313744187355042),
 ('great_crested_newt', 0.630035400390625),
 ('fritillary', 0.6287526488304138),
 ('yellowhead', 0.6284536719322205),
 ('tree_ferns', 0.6268918514251709),
 ('bettong', 0.6248616576194763),
 ('stag_beetles', 0.624174177646637),
 ('lady_slipper_orchids', 0.6238617300987244),
 ('carnivorous_marsupial', 0.6227748990058899),
 ('Powelliphanta', 0.6216369271278381),
 ('hoverfly', 0.619836688041687),
 ('sphinx_moth', 0.6191641092300415),
 ('nightjar', 0.6191612482070923),
 ('freshwater_crayfish', 0.6191293597221375),
 ('moray_eel', 0.6189029812812805),
 ('pipistrelle_bat', 0.6174764633178711),
 ('linnet', 0.6167510151863098),
 ('redshank', 0.6167505979537964),
 ('hawksbill_turtle', 0.6163536906242371),
 ('reed_warbler', 0.6144809126853943),
 ('weka', 0.6144742965698242),
 ('lady_slipper_orchid', 0.613978743553

In [70]:
autoextend.get_most_similar_lexemes('stag_beetle', rank_range=1000, num_word=1000, is_random=False)

[['constrictor', 'constrictor%1:05:00::'],
 ['viper', 'viper%1:05:00::'],
 ['boa', 'boa%1:05:00::'],
 ['python', 'python%1:05:00::'],
 ['snake', 'Snake%1:17:01::'],
 ['snake', 'snake%1:18:00::'],
 ['snake', 'Snake%1:17:00::'],
 ['snake', 'snake%2:38:01::'],
 ['snake', 'snake%2:42:00::'],
 ['pit_viper', 'pit_viper%1:05:00::'],
 ['water_moccasin', 'water_moccasin%1:05:01::'],
 ['rattlesnake', 'rattlesnake%1:05:00::'],
 ['python', 'Python%1:18:00::'],
 ['cobra', 'cobra%1:05:00::'],
 ['serpent', 'serpent%1:05:00::'],
 ['cottonmouth', 'cottonmouth_moccasin%1:05:00::'],
 ['boa_constrictor', 'boa_constrictor%1:05:00::'],
 ['lizard', 'lizard%1:05:00::'],
 ['gila_monster', 'Gila_monster%1:05:00::'],
 ['rodent', 'rodent%1:05:00::'],
 ['spider', 'spider%1:05:00::'],
 ['python', 'python%1:18:01::'],
 ['puff_adder', 'puff_adder%1:05:01::'],
 ['coral_snake', 'coral_snake%1:05:01::'],
 ['cat', 'cat%1:05:02::'],
 ['rat', 'rat%1:05:00::'],
 ['nonvenomous', 'nonvenomous%5:00:00:nontoxic:00'],
 ['copperh

In [61]:
design_targets = ['dryer%1:06:00::', 'refrigerator', 'chair', 'table']

stimulates = [
['stag_beetle', 'snail%2:35:00::'],
['stag_beetle', 'whelk%2:33:00::'],
['stag_beetle', 'raptorial%3:01:00::'],
['stag_beetle', 'jawless%3:01:00::'],
['stag_beetle', 'palmate%5:00:00:webbed:00'],
['stag_beetle', 'raptorial%5:00:00:aggressive:00'],

['snake', 'snake%2:38:01::'],
['snake', 'snake%2:42:00::'],
['snake', 'pup%2:29:00::'],
['snake', 'feline%3:01:00::'],
['snake', 'bulldog%3:01:00::'],
['snake', 'earless%3:00:00::'],
['snake', 'nonvenomous%5:00:00:nontoxic:00'],
['snake', 'barky%5:00:00:rough:00'],

['wave', 'wave%2:38:00::'],
['wave', 'tide%2:38:01::'],
['wave', 'surge%2:38:03::'],
['wave', 'agitated%3:00:02::'],
['wave', 'tidal%3:01:00::'],
['wave', 'stormy%3:00:00::'],

['sun', 'sun%2:39:00::'],
['sun', 'sunbathe%2:29:00::'],
['sun', 'solarize%2:39:00::'],
['sun', 'sunburn%2:29:00::'],
['sun', 'moonless%3:00:00::'],
['sun', 'starry%3:00:00::'],
['sun', 'moonlit%3:00:00::'],
['sun', 'light%3:00:06::']
]

analogies = []
for stimulate in stimulates:
    for design_target in design_targets:
        analogy = autoextend.get_analogy(stimulate[1], stimulate[0], design_target, is_single=False)
        if analogy is not None:
            analogies.append([f'{stimulate[1]} - {stimulate[0]} + {design_target}', analogy])

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [62]:
for i in analogies:
    print(i[0])
    print(i[1])

snail%2:35:00:: - stag_beetle + dryer%1:06:00::
['dryer', 'clothes_dryer', 'hair_dryer', 'blow_dryer', 'washing_machine', 'washer', 'blower', 'dishwasher', 'stove', 'water_heater']
snail%2:35:00:: - stag_beetle + refrigerator
['refrigerator', 'fridge', 'freezer', 'cooler', 'icebox', 'refrigerated', 'toaster_oven', 'dishwasher', 'dishwasher', 'dishwashing_machine']
snail%2:35:00:: - stag_beetle + chair
['chair', 'folding_chair', 'cart', 'cart', 'wheelchair', 'motorized_wheelchair', 'lawn_chair', 'chair', 'rocking_chair', 'chair']
snail%2:35:00:: - stag_beetle + table
['eucalyptus', 'sorrel', 'table', 'slender', 'free', 'feather', 'motley', 'burn', 'plume', 'hunchback']
whelk%2:33:00:: - stag_beetle + dryer%1:06:00::
['dryer', 'clothes_dryer', 'hair_dryer', 'blow_dryer', 'washing_machine', 'blower', 'washer', 'stove', 'dishwasher', 'water_heater']
whelk%2:33:00:: - stag_beetle + refrigerator
['refrigerator', 'fridge', 'freezer', 'cooler', 'icebox', 'refrigerated', 'crock_pot', 'toaster_o

In [63]:
design_targets = ['dryer%1:06:00::', 'refrigerator', 'chair', 'table']

stimulates = [
['stag_beetle', 'scissors'],

['snake', 'bellows'],
['snake', 'creeping'],

['wave', 'vibration'],

['sun', 'heat'],
['sun', 'light'],
]

analogies = []
for stimulate in stimulates:
    for design_target in design_targets:
        analogy = autoextend.get_analogy(stimulate[1], stimulate[0], design_target, is_single=False)
        if analogy is not None:
            analogies.append([f'{stimulate[1]} - {stimulate[0]} + {design_target}', analogy])

In [64]:
for i in analogies:
    print(i[0])
    print(i[1])

scissors - stag_beetle + dryer%1:06:00::
['dryer', 'clothes_dryer', 'blow_dryer', 'hair_dryer', 'blower', 'washing_machine', 'washer', 'curling_iron', 'dishwasher', 'washer']
scissors - stag_beetle + refrigerator
['refrigerator', 'fridge', 'freezer', 'dishwasher', 'dishwasher', 'dryer', 'cooler', 'scissors', 'toaster_oven', 'washer']
scissors - stag_beetle + chair
['chair', 'scissors', 'scissors', 'gurney', 'shellfish', 'folding_chair', 'barber_chair', 'cart', 'cart', 'lawn_chair']
scissors - stag_beetle + table
['frog', 'sorrel', 'eucalyptus', 'shellfish', 'scissors', 'roundworm', 'scissors', 'hunchback', 'motley', 'plume']
bellows - snake + dryer%1:06:00::
['dryer', 'bellows', 'blower', 'blow_dryer', 'blower', 'clothes_dryer', 'hair_dryer', 'device', 'washing_machine', 'tumble_drier']
bellows - snake + refrigerator
['bellows', 'refrigerator', 'fridge', 'blower', 'device', 'dishpan', 'casement_window', 'firebox', 'blower', 'thermos_bottle']
bellows - snake + chair
['bellows', 'blower'

In [ ]:
whelk%2:33:00::

In [39]:
vector = autoextend.get_verctor_from_word('woman', synset_idx=2)
np.linalg.norm(vector, ord=2)

a human female employed to do housework


0.40453792896463986

In [ ]:
autoextend.get_analogy('dryer', )

類推考察

In [36]:
vector = autoextend.get_verctor_from_word('woman', synset_idx=-1)
[i[0] for i in autoextend.get_word_from_vector(vector)]

women as a class


['woman',
 'woman',
 'woman',
 'man',
 'womanhood',
 'man',
 'man',
 'female',
 'girl',
 'man']

In [53]:
import pandas as pd

def analogy_to_csv(words_list):
    df = pd.DataFrame(None)
    for i, words in enumerate(words_list):
        word1, word2, word3 = words

        # AutoExtend
        output_list = autoextend.get_analogy(word1, word2, word3)
        for j, output in enumerate(output_list):
            if j <= 3:
                j = 0
            else:
                j -= 3
            row = [i] + [j] + ['autoextend'] + [output[0]] + output[1]
            series = pd.Series(row)
            df = df.append(series, ignore_index=True)

        # word2vec
        output_words = word_embed.most_similar(positive=[word1, word3], negative=[word2])
        output_words = [word for word, _ in output_words]
        for word in words:
            row = [i] + [0] + ['word2vec'] + [word] + word_embed.word_vec(word).tolist()
            series = pd.Series(row)
            df = df.append(series, ignore_index=True)

        for j, word in enumerate(output_words):
            row = [i] + [j+1] + ['word2vec'] + [word] + word_embed.word_vec(word).tolist()
            series = pd.Series(row)
            df = df.append(series, ignore_index=True)
        
        df[0] = df[0].astype(int)
        df[1] = df[1].astype(int)
        
    df.to_csv('analogy_experiment.csv')

words_list =  [['king', 'man', 'woman'], ['queen', 'woman', 'man'], ['brother', 'man', 'woman'], ['easily', 'easy', 'quick'], ['better', 'good', 'bad']]
analogy_to_csv(words_list)

/var/folders/xl/d7b96jtx11935vw0d7m2sj0r0000gn/T/ipykernel_33977/2955068889.py:23: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  row = [i] + [0] + ['word2vec'] + [word] + word_embed.word_vec(word).tolist()
/var/folders/xl/d7b96jtx11935vw0d7m2sj0r0000gn/T/ipykernel_33977/2955068889.py:28: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  row = [i] + [j+1] + ['word2vec'] + [word] + word_embed.word_vec(word).tolist()


In [ ]:
words_list = [['king', 'man', 'woman'], ['queen', 'woman', 'man'], ['brother', 'man', 'woman'], ['easily', 'easy', 'quick'], ['better', 'good', 'bad']]

In [ ]:
words_list = [['king', 'man', 'woman'], ['easily', 'easy', 'quick']]
analogy_to_csv(words_list)

In [21]:
autoextend.get_analogy('easily', 'easy', 'quick')

easily - easy + quick = well


'well'

In [20]:
word_embed.most_similar(positive=['easily', 'quick'], negative=['easy'])

[('rapidly', 0.5291780233383179),
 ('quickly', 0.46909627318382263),
 ('swiftly', 0.44768333435058594),
 ('rapidity', 0.41088029742240906),
 ('readily', 0.3970886766910553),
 ('nimbly', 0.3895303010940552),
 ('explosively', 0.38298866152763367),
 ('speedily', 0.38134026527404785),
 ('subprime_mortgage_resets', 0.3653566241264343),
 ('successfully', 0.3618415296077728)]

In [3]:
def analogy_by_similarity(word):
    idx1, idx2, idx3  = rand_ints_nodup(0, 9, 3)
    words = word_embed.most_similar([word])
    results = word_embed.most_similar(positive=[words[idx1][0], words[idx2][0]],negative=[words[idx3][0]])
    output_word = None
    for result_word in results:
        if (result_word != words[idx1][0]) and \
           (result_word != words[idx2][0]) and \
           (result_word != words[idx3][0]):
           
            output_word = result_word
            break
        
    print(output_word)

In [16]:
def analogy_by_hypo_hyper(word, synset_idx=0):
    sensekey_input = autoextend.get_senseky_from_word(word, synset_idx=0)
    similar1 = autoextend.get_most_similar_lexemes(word, synset_idx=0)[1]
    similar2 = autoextend.get_most_similar_lexemes(word, synset_idx=0)[1]
    similar3 = autoextend.get_most_similar_lexemes(word, synset_idx=0)[1]
    hyper = get_hypernum(word, idx=synset_idx)[1]
    hyper_hyper = get_hypernum(get_hypernum(word, idx=synset_idx)[0])[1]
    hypo1 = get_hyponym(word, idx=synset_idx)[1]
    hypo2 = get_hyponym(word, idx=synset_idx)[1]
    hypo3 = get_hyponym(word, idx=synset_idx)[1]

    word_list = [sensekey_input, similar1, similar2, similar3, hyper, hyper_hyper, hypo1, hypo2, hypo3]
    
    for i in word_list:
        if i is not None:
            print(i)
        else:
            print('sensekey is None')

    for _ in range(20):
        idx1, idx2, idx3 = rand_ints_nodup(0, len(word_list)-1, 3)
        autoextend.get_analogy_by_sensekey(word_list[idx1], word_list[idx2], word_list[idx3])

    analogy_words = []
    analogy_words.append(autoextend.get_analogy_by_sensekey(hyper, sensekey_input, similar1))
    analogy_words.append(autoextend.get_analogy_by_sensekey(hyper, sensekey_input, similar2))
    analogy_words.append(autoextend.get_analogy_by_sensekey(sensekey_input, hypo1, similar1))
    analogy_words.append(autoextend.get_analogy_by_sensekey(sensekey_input, hypo1, similar2))
    analogy_words.append(autoextend.get_analogy_by_sensekey(sensekey_input, hypo2, similar1))
    analogy_words.append(autoextend.get_analogy_by_sensekey(sensekey_input, hypo2, similar2))
    analogy_words.append(autoextend.get_analogy_by_sensekey(hyper_hyper, sensekey_input, similar1))
    analogy_words.append(autoextend.get_analogy_by_sensekey(hyper_hyper, sensekey_input, similar2))

    


In [17]:
def analogy_by_hypo_hyper_other(word1, word2, synset_idx=0):
    sensekey_input1 = autoextend.get_senseky_from_word(word1, synset_idx=0)
    sensekey_input2 = autoextend.get_senseky_from_word(word2, synset_idx=0)
    similar1 = autoextend.get_most_similar_lexemes(word1, synset_idx=0)[1]
    similar2 = autoextend.get_most_similar_lexemes(word1, synset_idx=0)[1]
    hyper = get_hypernum(word1, idx=synset_idx)[1]
    hyper_hyper = get_hypernum(get_hypernum(word1, idx=synset_idx)[0])[1]
    hyper_hyper_hyper = get_hypernum(get_hypernum(get_hypernum(word1, idx=synset_idx)[0])[0])[1]
    hypo1 = get_hyponym(word1, idx=synset_idx)[1]
    hypo2 = get_hyponym(word1, idx=synset_idx)[1]
    
    print(sensekey_input1, sensekey_input2 ,similar1, similar2, hyper, hyper_hyper, hypo1, hypo2)

    autoextend.get_analogy_by_sensekey(hyper, sensekey_input1, sensekey_input2)
    autoextend.get_analogy_by_sensekey(sensekey_input1, hypo1, sensekey_input2)
    autoextend.get_analogy_by_sensekey(sensekey_input1, hypo2, sensekey_input2)
    autoextend.get_analogy_by_sensekey(hyper_hyper, sensekey_input1, sensekey_input2)

    autoextend.get_analogy_by_sensekey(hyper, sensekey_input2, sensekey_input1)
    autoextend.get_analogy_by_sensekey(sensekey_input2, hypo1, sensekey_input1)
    autoextend.get_analogy_by_sensekey(sensekey_input2, hypo2, sensekey_input1)
    autoextend.get_analogy_by_sensekey(hyper_hyper, sensekey_input2, sensekey_input1)

    print('hyper____')
    autoextend.get_analogy_by_sensekey(hyper_hyper_hyper, sensekey_input1, sensekey_input2)
    autoextend.get_analogy_by_sensekey(hyper_hyper_hyper, sensekey_input1, hypo1)
    autoextend.get_analogy_by_sensekey(hyper_hyper_hyper, sensekey_input2, hypo2)
    autoextend.get_analogy_by_sensekey(hyper_hyper_hyper, sensekey_input2, similar1)
    autoextend.get_analogy_by_sensekey(hyper_hyper_hyper, sensekey_input2, similar2)


In [18]:
analogy_by_hypo_hyper_other('table', 'beauty')

table%1:14:00:: beauty%1:07:00:: table%1:06:01:: contents%1:10:00:: array%1:14:00:: arrangement%1:14:00:: contents%1:10:00:: actuarial_table%1:14:00::
array - table + beauty = loveliness
table - contents + beauty = glamor
None
arrangement - table + beauty = loveliness
array - beauty + table = slender
beauty - contents + table = glamor
None
arrangement - beauty + table = slender
hyper____
planning - table + beauty = loveliness
planning - table + contents = contained
None
planning - beauty + table = slender
planning - beauty + contents = table


In [19]:
analogy_by_hypo_hyper_other('dryer', 'fun')

dryer%1:06:00:: fun%1:04:00:: washing_machine%1:06:00:: stove%1:06:00:: appliance%1:06:00:: device%1:06:00:: clothes_dryer%1:06:00:: clothes_dryer%1:06:00::
appliance - dryer + fun = durables
dryer - clothes_dryer + fun = hair_dryer
dryer - clothes_dryer + fun = hair_dryer
device - dryer + fun = carpet
appliance - fun + dryer = clothes_dryer
fun - clothes_dryer + dryer = hair_dryer
fun - clothes_dryer + dryer = hair_dryer
device - fun + dryer = clothes_dryer
hyper____
instrumentality - dryer + fun = bag
instrumentality - dryer + clothes_dryer = unlawfully
instrumentality - fun + clothes_dryer = dryer
instrumentality - fun + washing_machine = washer
instrumentality - fun + stove = cooker


In [20]:
analogy_by_hypo_hyper_other('dryer', 'strange')

dryer%1:06:00:: strange%3:00:00:: water_heater%1:06:00:: microwave_oven%1:06:00:: appliance%1:06:00:: device%1:06:00:: clothes_dryer%1:06:00:: hand_blower%1:06:00::
appliance - dryer + strange = durables
dryer - clothes_dryer + strange = hair_dryer
None
device - dryer + strange = carpet
appliance - strange + dryer = clothes_dryer
strange - clothes_dryer + dryer = hair_dryer
None
device - strange + dryer = clothes_dryer
hyper____
instrumentality - dryer + strange = preserver
instrumentality - dryer + clothes_dryer = unlawfully
None
instrumentality - strange + water_heater = stove
instrumentality - strange + microwave_oven = microwave


In [21]:
def compare_analogy(word, synset_idx=0):
    print('####    類似度の類推     #####')
    for _ in range(10):
        analogy_by_similarity(word)
    
    print('####    上位下位の類推     #####')
    analogy_by_hypo_hyper(word, synset_idx=synset_idx)

In [22]:
compare_analogy('suit', synset_idx=0)

####    類似度の類推     #####
('Complaint', 0.5637338161468506)
('Mustard_pakki', 0.5386776924133301)
('Al_Edhari', 0.49727824330329895)
('pompadoured_hair', 0.6859167814254761)
('lawsuit', 0.671337366104126)
('pompadoured_hair', 0.6738342642784119)
('lawsuits_alleging', 0.5511248707771301)
('lawsuit', 0.5822794437408447)
('alleging', 0.5666015148162842)
('suit', 0.5710189938545227)
####    上位下位の類推     #####
suit%1:06:00::
gabardine%1:06:03::
zoot_suit%1:06:00::
suit%1:06:01::
garment%1:06:00::
clothing%1:06:00::
zoot_suit%1:06:00::
slack_suit%1:06:00::
double-breasted_suit%1:06:00::
None
None
None
None
suit - suit + gabardine = trousers
None
None
None
clothing - garment + suit = apparel
zoot_suit - clothing + suit = porkpie_hat
None
None
None
None
suit - zoot_suit + garment = screamer
gabardine - zoot_suit + garment = screamer
None
None
None
None
garment - suit + gabardine = trousers
garment - suit + zoot_suit = jacket
suit - zoot_suit + gabardine = screamer
suit - zoot_suit + zoot_suit = 

In [10]:
compare_analogy('table', synset_idx=1)

####    類似度の類推     #####
('trays', 0.5813458561897278)
('Ethan_Hession', 0.44691479206085205)
('wine_goblet', 0.5873473286628723)
('drill_sergeant_Perkovic', 0.47566887736320496)
('reconciles_EBITDA', 0.44722437858581543)
('dinning_room', 0.608176052570343)
('reconciles_EBITDA', 0.44722437858581543)
('drill_sergeant_Perkovic', 0.45419415831565857)
('toasted_cashews', 0.4707796573638916)
('tables', 0.5040887594223022)
####    上位下位の類推     #####
table%1:14:00::
table%1:14:01::
tray%1:06:00::
drawer%1:18:01::
furniture%1:06:00::
furnishing%1:06:00::
stand%1:06:04::
conference_table%1:06:00::
tea_table%1:06:00::
None
None
table - drawer + tray = toast
None
None
table - tray + table = draft
None
stand - drawer + furniture = cobblestone
None
None
furniture - table + table = sideboard
furniture - table + tray = turntable
table - stand + table = club
table - stand + tray = lazy_susan
None
None
None
None


In [17]:
compare_analogy('wheel', synset_idx=0)

####    類似度の類推     #####
('Nancy_Pekarek_spokeswoman', 0.505904495716095)
('stealth_removable', 0.580708384513855)
('rider_Shinichi_Nakatomi', 0.5683863162994385)
('No.##_Roush_Racing', 0.5009022951126099)
('Rear_wheel', 0.5558632612228394)
('footpeg', 0.609529435634613)
####    上位下位の類推     #####
wheel%1:06:00:: nosewheel%1:06:00:: paddlewheel%1:06:00:: machine%1:06:02:: device%1:06:00:: wagon_wheel%1:06:00:: daisy_print_wheel%1:06:00::
None
None
None
None
None
None
device - nosewheel + paddlewheel = sternwheeler
None
nosewheel - device + wheel = landing_gear
None
machine - wheel + nosewheel = tailplane
machine - wheel + paddlewheel = sternwheeler
None
None
None
None
device - wheel + nosewheel = tailplane
device - wheel + paddlewheel = sternwheeler


In [18]:
compare_analogy('tire', synset_idx=1)

####    類似度の類推     #####
('tire', 0.6417492032051086)
('tire', 0.6120626330375671)
('beadlock', 0.583296000957489)
('Pirelli_tire', 0.5449734926223755)
('Corteco', 0.5167824625968933)
('tire', 0.5241057872772217)
####    上位下位の類推     #####
tire%1:06:00:: tubeless_tire%1:06:00:: tubeless_tire%1:06:00:: devolve%2:29:00:: delegating%1:04:00:: retire%2:37:00:: poop_out%2:29:00::
None
retire - tubeless_tire + tubeless_tire = withdraw
None
tubeless_tire - delegating + tubeless_tire = tubeless
None
None
None
None
retire - tubeless_tire + tire = connexion
None
None
None
tire - retire + tubeless_tire = pneumatic_tire
tire - retire + tubeless_tire = pneumatic_tire
None
None
delegating - tire + tubeless_tire = decentralizing
delegating - tire + tubeless_tire = decentralizing


In [33]:
compare_analogy('chair')

####    類似度の類推     #####
('Co_Chair', 0.6625238060951233)
('Chairs', 0.6326252222061157)
('chairperson', 0.6012726426124573)
('chair', 0.6340569853782654)
('cochairs', 0.5543997287750244)
('Chairwoman', 0.7123143672943115)
('Co_Chair', 0.6384541392326355)
('Chair_Elect', 0.60837721824646)
('Chairperson', 0.6735294461250305)
('chaired', 0.555755078792572)
####    上位下位の類推     #####
chair%1:06:00:: stool%1:06:00:: chaise_longue%1:06:00:: seat%1:06:01:: space%1:15:00:: folding_chair%1:06:00:: straight_chair%1:06:00::
None
None
None
None
chair - barber_chair + chaise_longue = chaise
None
chaise_longue - armchair + space = chaise
space - folding_chair + chair = livingroom
None
None
seat - chair + stool = footstool
seat - chair + chaise_longue = chaise
chair - folding_chair + stool = milking_stool
chair - folding_chair + chaise_longue = chaise
None
None
space - chair + stool = harborage
space - chair + chaise_longue = chaise


In [20]:
compare_analogy('suit')

####    類似度の類推     #####
('pompadoured_hair', 0.6018907427787781)
('alleging', 0.580407977104187)
('Wearing_beige', 0.5911272764205933)
('Al_Edhari', 0.47900304198265076)
('lawsuit', 0.5446116328239441)
('sported_bushy_reddish_sideburns', 0.5645790100097656)
####    上位下位の類推     #####
suit%1:06:00:: tuxedo%1:06:00:: trousers%1:06:00:: garment%1:06:00:: clothing%1:06:00:: slack_suit%1:06:00:: slack_suit%1:06:00::
None
None
None
garment - clothing + trousers = jacket
None
None
clothing - garment + tuxedo = tux
trousers - garment + clothing = clothes
None
None
garment - suit + tuxedo = tux
garment - suit + trousers = skirt
None
None
None
None
clothing - suit + tuxedo = tux
clothing - suit + trousers = clothes


In [ ]:
analogy_by_hypo_hyper_other('beauty', 'table')

beauty%1:07:00:: prettiness%1:07:00:: loveliness%1:07:00:: appearance%1:07:00:: quality%1:07:00:: pulchritude%1:07:00:: pulchritude%1:07:00::
appearance beauty beauty
['slender', 'art', 'light', 'light', 'light', 'clear', 'light', 'mercantilism', 'shape', 'mug']
appearance - beauty + beauty = slender
appearance beauty beauty
['slender', 'art', 'light', 'light', 'light', 'clear', 'light', 'mercantilism', 'shape', 'mug']
appearance - beauty + beauty = slender
beauty pulchritude pulchritude
['forward', 'blow', 'blow', 'have', 'turn', 'sheer', 'fresh', 'blow', 'inflation', 'heavy']
beauty - pulchritude + pulchritude = forward
beauty pulchritude pulchritude
['forward', 'blow', 'blow', 'have', 'turn', 'sheer', 'fresh', 'blow', 'inflation', 'heavy']
beauty - pulchritude + pulchritude = forward
beauty pulchritude pulchritude
['forward', 'blow', 'blow', 'have', 'turn', 'sheer', 'fresh', 'blow', 'inflation', 'heavy']
beauty - pulchritude + pulchritude = forward
beauty pulchritude pulchritude
['f

In [10]:
analogy_by_hypo_hyper_other('beauty', 'chair')

beauty%1:07:00:: loveliness%1:07:00:: comeliness%1:07:00:: appearance%1:07:00:: quality%1:07:00:: picturesqueness%1:07:00:: handsomeness%1:07:00::
appearance beauty beauty
['chair', 'swivel_chair', 'armchair', 'overstuffed_chair', 'highchair', 'recliner', 'rocking_chair', 'wheelchair', 'reclining_chair', 'seat']
appearance - beauty + beauty = chair
appearance beauty beauty
['chair', 'swivel_chair', 'armchair', 'overstuffed_chair', 'highchair', 'recliner', 'rocking_chair', 'wheelchair', 'reclining_chair', 'seat']
appearance - beauty + beauty = chair
beauty picturesqueness picturesqueness
None
beauty picturesqueness picturesqueness
None
beauty handsomeness handsomeness
['heavy', 'chair', 'quilt', 'bookfair', 'crochet', 'mat', 'peoria', 'picnic', 'heavy', 'fair']
beauty - handsomeness + handsomeness = heavy
beauty handsomeness handsomeness
['heavy', 'chair', 'quilt', 'bookfair', 'crochet', 'mat', 'peoria', 'picnic', 'heavy', 'fair']
beauty - handsomeness + handsomeness = heavy
quality bea

NameError: name 'compare_analogy' is not defined